# Sprint 5 - Inferencia estadística
#### Autor: Nicolás Núñez

In [1]:
# Librerias requeridas
import math
import pandas as pd
import numpy as np
import pingouin as pg
import scipy.stats as stats

### Ejercicio 1


Cierta compañía aérea ha requerido de sus servicios profesionales para
llevar a cabo un estudio relacionado con la cantidad de pasajeros que llegan
al país en vuelos internacionales, dicho estudio tiene relación con la
enfermedad de COVID-19. De acuerdo a la información que maneja la
compañía, la cantidad de pasajeros que llegan en vuelos internacionales se
comporta aproximadamente normal con una media de 120 pasajeros y una
varianza de 25.

1. Asumiendo que cada día llegan a Santiago 16 vuelos internacionales,
determine la probabilidad que durante el año (52 semanas) la cantidad
total de pasajeros sea de al menos de 699500.


**Respuesta**: Considerando que la cantidad por vuelo de pasajeros se distribuye $X \sim N(120,25^2)$, entonces la cantidad anual considerando 16 vuelos diarios y 52 semanas en año queda distribuida por $Y=16*7*52*X \sim N((16*7*52)*120, (16*7*52)^2*25^2)$, y de esta forma nos interesa calcular:
$$P(Y\geq 699500)$$
De esta forma considerando que
$$\mu_Y = (16*7*52)*120 =698880\quad\text{y}\quad \sigma_Y = (16*7*52)*25 = 145600$$
podemos encontrar el $z$-score para $Y=699500$, el cual queda dado por
$$z = \dfrac{699500 - 698880}{145600}\approx 0.004$$
con lo cual, con ayuda de la libreria scipy es posible, se sigue que 
$$P(Y\geq 699500) = P(z\geq 0.004) = 0.4984$$
es decir, que la probabilidad de que la cantidad total de pasajeros sea de al menos 699500 personas es de $49.84$% aproximadamente.

In [2]:
print(f"P(z>=0.004) = {1-stats.norm.cdf(0.004)}.")

P(z>=0.004) = 0.4984042351337684.


2. De acuerdo con registros históricos se sabe que aproximadamente un
7% de los pasajeros son niños. Considerando un potencial de 400
pasajeros que llegarían en los próximos días, determine la
probabilidad que menos de un 5% sean niños.

**Respuesta**: Considerando el enunciado, podemos considerar que la cantidad de niños entre los 400 pasajeros se modela a través de una distribución binomial con probabilidad 7%, esto es $C \sim bin(400, 0.07)$, de esta forma, la probabilidad de que menos el 5% sean niños queda expresada por:
$$P(C< 400*0.05 = 20) = \sum_{k=0}^{19} \begin{pmatrix} 400\\ k\end{pmatrix}0.07^k(1-0.07)^{400-k}$$
Realizamos los cálculos mencionados a continuación:

In [3]:
# Datos
total = 400
n = 19
p = 0.07
prob = 0

# Sumatoria
for k in range(n+1):
    prob += math.comb(total,k)*(p**k)*((1-p)**(total-k))

print(f"La probabilidad de que entre los {total} pasajeros hayan menos de {n+1} niños es de {prob*100:.2f}%.")

La probabilidad de que entre los 400 pasajeros hayan menos de 20 niños es de 4.22%.


### Ejercicio 2


Una empresa que se dedica a hacer encuestas se queja de que un agente
realiza en promedio 53 encuestas por semana. Se ha introducido una forma
más moderna de realizar las encuetas y la empresa quiere evaluar su
efectividad. Los números de encuestas realizadas en una semana por una
muestra aleatoria de agentes son:
$$\begin{array}{|c|c|c|c|c|c|c|c|c|c|}
\hline
53 & 57 & 50 & 55 & 58 & 54 & 60 & 52 & 59 & 62\\
\hline
60  & 60 & 51 & 59 & & & & & & \\
\hline
\end{array}
$$
En el nivel de significancia 0,05, puede concluirse que la cantidad media de
entrevistas realizadas por los agentes es superior a 53 por semana? Evalúe
el valor p

**Respuesta**: Consideremos las hipótesis
$$H_0 : \mu \leq 53$$
$$H_1 : \mu > 53$$
para la cual se considerará una significancia de $\alpha=0.05$ la muestra tabulada. Se procederá implementando dicha prueba de hipótesis teniendo en cuenta que no se conoce el valor de $\sigma$, y por lo tanto el test se realizará con el estadístico T-student:

In [4]:
# Datos
muestra = [53,57,50,55,58,54,60,52,59,62,60,60,51,59]
mu = 53
alpha = 0.05

# Prueba de normalidad
print("Prueba de normalidad de los datos:")
display(pg.normality(data=muestra))

# Prueba T unilateral
t, p = stats.ttest_1samp(muestra, mu, alternative='greater')

# Evaluación hipótesis nula
if t>= stats.t.ppf(1-alpha,len(muestra)-1):
    print("Rechazamos la hipótesis nula.")
else:
    print("No hay suficiente evidencia para rechazar la hipótesis nula.")

Prueba de normalidad de los datos:


,W,pval,normal
0,0.925878,0.266872,True


Rechazamos la hipótesis nula.


Es decir, rechazamos la hipotesis nula, concluyendo entonces con la significancia pedida que la cantidad media de
entrevistas realizadas por los agentes es superior a 53. 

### Ejercicio 3

En el último año, se determinó que el tiempo promedio que los estudiantes
pasaron resolviendo un examen fue de 65 minutos, con una desviación
estándar de 10 minutos. Este año, se seleccionó una muestra aleatoria de 50
estudiantes, y se encontró que el tiempo promedio que tomaron resolviendo
el mismo examen fue de 68 minutos.

Considerando que el objetivo es evaluar si hay evidencia suficiente para
afirmar que el tiempo promedio ha aumentado este año en comparación con
el último, se plantea la siguiente prueba de hipótesis con un nivel de
significancia del 5%:

- **Hipótesis nula($H_0$)**:  El tiempo promedio este año ($\mu$) es igual al tiempo
promedio del año pasado ($\mu_0$): $\mu = \mu_0$.

- **Hipótesis alternativa($H_1$)**: El tiempo promedio este año $(\mu)$ es mayor que el proimedio del año pasado $(\mu_0)$: $\mu>\mu_0$.

Se dispone de la desviación estándar poblacional ($\sigma$) del año pasado, lo que
permite utilizar una prueba Z para la media poblacional.

1. Calcule el estadístico de prueba Z utilizando los datos proporcionados.

**Respuesta**: Primero calculamos el estadístico $Z$ considerando la expresión que lo defina como
$$Z = \dfrac{\overline{x}-\mu}{\frac{\sigma}{\sqrt{n}}}$$
con
- $\overline{x}$: Media de la muestra.
- $\mu$: Media poblacional de la hipotesis nula.
- $\sigma$: Desviación estándar poblacional.
- $n$: Tamaño de la muestra.
Para nuestro caso, 
$$\overline{x} = 68,\quad \mu = 65,\quad \sigma = 10,\quad n=50$$
De esta forma, procedemos a realizar la implementación:

In [5]:
# Datos
x_bar = 68
mu = 65
sigma = 10
n = 50

# Estadístico de prueba Z
z = (x_bar - mu)/(sigma/(n**.5))

print(f"El estadístico Z utilizando los datos proporcionados queda dado por Z={z:.4f}.")


El estadístico Z utilizando los datos proporcionados queda dado por Z=2.1213.


2. ¿Qué conclusión se puede obtener con respecto a las hipótesis nula y
alternativa basándose en el valor calculado de Z y el valor crítico
correspondiente para un nivel de significancia del 5%?

**Respuesta**: Considerando la prueba de hipótesis con un nivel de significancia de un 5% se tiene lo siguiente:

In [6]:
# Significancia
alpha = 0.05

# Valor critico para prueba de una cola por derecha
vc = stats.norm.ppf(1-alpha)

# Evaluar la hipótesis nula
if z>vc:
    print("Rechazamos la hipótesis nula.")
else:
    print("No hay suficiente evidencia para rechazar la hipótesis nula.")   


Rechazamos la hipótesis nula.


Es decir, rechazamos la hipotesis nula, concluyendo entonces con la significancia pedida que la media de este año es mayor que la del año anterior. 